## Driver

In [1]:
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.drivers.qmolecule import QMolecule

# molecule = "H .0 .0 .0; H .0 .0 0.739"
molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PySCFDriver(atom = molecule)
qmolecule = driver.run()

## Electronic Structure and Freeze Core Qubits and Remove Orbitals

In [8]:
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers import FreezeCoreTransformer

problem = ElectronicStructureProblem(driver, q_molecule_transformers = [FreezeCoreTransformer(freeze_core = True)])

second_quantized_operator = problem.second_q_ops()

# Main Operator = Hamiltonian
main_operator = second_quantized_operator[0]

# print(main_operator)

In [9]:
# data = problem.molecule_data_transformed
# print(data)

## Qubit Converter (Mapping) and Z2Symmetry Reduction

In [10]:
from qiskit_nature.mappers.second_quantization import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

# mapping = "JordanWignerMapper"
mapping = "ParityMapper"
# mapping = "BravyiKitaevMapper"

if mapping == "ParityMapper":
    mapping = ParityMapper()
elif mapping == "JordanWignerMapper":
    mapping = JordanWignerMapper()
elif mapping == "BravyiKitaevMapper":
    mapping = BravyiKitaevMapper()
    



# converter = QubitConverter(mapper = mapping, two_qubit_reduction = False)
# converter = QubitConverter(mapper = mapping, two_qubit_reduction = True, z2symmetry_reduction = [1, 1, 1, 1])
converter = QubitConverter(mapper = mapping, two_qubit_reduction = True, z2symmetry_reduction = [1, 1])

# Possible Z2Symmetries 

# [1, 1, 1, 1], [1, 1, 1, -1], [1, 1, -1, 1], 
# [1, 1, -1, -1], [1, -1, 1, 1], [1, -1, 1, -1], 
# [1, -1, -1, 1], [1, -1, -1, -1], [-1, 1, 1, 1], [-1, 1, 1, -1], 
# [-1, 1, -1, 1], [-1, 1, -1, -1], [-1, -1, 1, 1], [-1, -1, 1, -1], 
# [-1, -1, -1, 1], [-1, -1, -1, -1]

In [11]:
# print(converter.z2symmetry_reduction)
# print(converter.two_qubit_reduction)
# print(converter.z2symmetries)

In [12]:
# Mapping fermionic operators to qubit operators
num_particles = (problem.molecule_data_transformed.num_alpha, problem.molecule_data_transformed.num_beta)

print(num_particles)

(1, 1)


In [13]:
qubit_operator = converter.convert(main_operator, num_particles = num_particles)
# print(qubit_operator)

## Z2Symmetries and Remove Orbitals

In [65]:
from qiskit.opflow.primitive_ops import Z2Symmetries

sym = Z2Symmetries.find_Z2_symmetries(qubit_operator)
print(sym)

Z2 symmetries:
Symmetries:
IIIIZIII
IIZZIIZZ
IZIIIZII
ZIIIIIII
Single-Qubit Pauli X:
IIIIXIII
IIIIIIIX
IIIIIXII
XIIIIIII
Cliffords:
0.7071067811865475 * IIIIZIII
+ 0.7071067811865475 * IIIIXIII
0.7071067811865475 * IIZZIIZZ
+ 0.7071067811865475 * IIIIIIIX
0.7071067811865475 * IZIIIZII
+ 0.7071067811865475 * IIIIIXII
0.7071067811865475 * ZIIIIIII
+ 0.7071067811865475 * XIIIIIII
Qubit index:
[3, 0, 2, 7]
Tapering values:
  - Possible values: [1, 1, 1, 1], [1, 1, 1, -1], [1, 1, -1, 1], [1, 1, -1, -1], [1, -1, 1, 1], [1, -1, 1, -1], [1, -1, -1, 1], [1, -1, -1, -1], [-1, 1, 1, 1], [-1, 1, 1, -1], [-1, 1, -1, 1], [-1, 1, -1, -1], [-1, -1, 1, 1], [-1, -1, 1, -1], [-1, -1, -1, 1], [-1, -1, -1, -1]


In [100]:
# remove_ob = FreezeCoreTransformer.transform(True, molecule_data = qubit_operator)
# print(remove_ob)

In [6]:
# from qiskit.opflow.primitive_ops import Z2Symmetries

# # sym = Z2Symmetries.find_Z2_symmetries(qubit_operator)
# sym = Z2Symmetries.symmetries
# print(sym)

## Initial State (Hartree Fock)

In [14]:
from qiskit_nature.circuit.library import HartreeFock

num_particles = (problem.molecule_data_transformed.num_alpha, problem.molecule_data_transformed.num_beta)

num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
print(num_spin_orbitals)

initial_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print(initial_state)

10
     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     └───┘
q_2: ─────
          
q_3: ─────
          
q_4: ─────
          
q_5: ─────
          


## Ansatz

In [15]:
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

# Available Ansatz

# ansatz = "TwoLocal"
ansatz = "UCCSD"
  
if ansatz == "UCCSD":
    ansatz = UCCSD(qubit_converter = converter, 
                   num_particles = num_particles, 
                   num_spin_orbitals = num_spin_orbitals, 
                   initial_state = initial_state)
elif ansatz == "PUCCD":
    ansatz = PUCCD(qubit_converter = converter, 
                   num_particles = num_particles, 
                   num_spin_orbitals = num_spin_orbitals, 
                   initial_state = initial_state)
elif ansatz == "SUCCD":
    ansatz = SUCCD(qubit_converter = converter, 
                   num_particles = num_particles, 
                   num_spin_orbitals = num_spin_orbitals, 
                   initial_state = initial_state)

elif ansatz == "TwoLocal":
    # Ansatz Parameters
    # Single qubit rotations
    rotation_blocks = ['ry', 'rz']
    
    # Entanglement gates
    entanglement_blocks = 'cx'
    
    # Type of entanglement
    entanglement = "full"    # or "linear"
    
    # Repitition of ansatz
    repititions = 6    # arbitrary number
    
    # Skip final rotation layer
    skip_final_rotation_layer = True
    
    # Ansatz
    ansatz = TwoLocal(num_qubits = qubit_operator.num_qubits, 
                      rotation_blocks = rotation_blocks, 
                      entanglement_blocks = entanglement_blocks, 
                      entanglement = entanglement, 
                      reps = repititions, 
                      skip_final_rotation_layer = skip_final_rotation_layer)
    
    ansatz.compose(initial_state, front = True, inplace = True)

# elif ansatz == "Custom":
#     # Add custom ansatz

## Backend

In [16]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

## Optimizer

In [17]:
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

optimizer = "COBYLA"

if optimizer == "COBYLA":
    optimizer = COBYLA(maxiter = 500)
elif optimizer == "L_BFGS_B":
    optimizer = L_BFGS_B(maxiter = 500)
elif optimizer == "SPSA":
    optimizer = SPSA(maxiter = 500)
elif optimizer == "SLSQP":
    optimizer = SLSQP(maxiter = 500)

## VQE and Inital Parameters of Ansatz

In [18]:
from qiskit.algorithms import VQE
from IPython.display import display, clear_output

# Initial Parameters of Ansatz
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters
    
algorithm = VQE(ansatz = ansatz, 
                optimizer = optimizer, 
                initial_point = initial_point, 
                quantum_instance = backend)
#                 callback = callback)

result = algorithm.compute_minimum_eigenvalue(qubit_operator)
print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 61,
    'eigenstate': array([ 3.54535463e-17-5.92271642e-18j,  3.95059244e-17+1.23125663e-16j,
        4.79814338e-16-4.03377094e-17j,  9.92852929e-01-5.31670101e-16j,
        3.63531253e-17+1.19052598e-18j, -5.60393291e-18+2.49047340e-18j,
       -1.00178753e-02+7.01649221e-17j, -1.93728743e-16-2.15064436e-16j,
       -4.42274364e-17+2.14841622e-17j, -2.23559047e-17+3.50090602e-17j,
       -8.15569095e-17+5.50858311e-17j, -2.28313243e-16-4.65338287e-17j,
       -3.19682615e-17+3.67268245e-17j,  3.22288095e-18+1.01096142e-17j,
       -2.99985159e-02+3.11661780e-17j,  4.11402314e-17+8.99610572e-17j,
       -1.38871220e-17+1.53148798e-17j,  1.94817207e-17-2.82136940e-17j,
        2.99080220e-17-2.98856808e-17j,  1.13061985e-16+9.57060180e-17j,
       -1.64925592e-17-8.12941988e-18j, -2.57232206e-17+2.26810714e-17j,
       -1.41048771e-17+5.98058369e-17j, -7.63628788e-18-4.29843763e-17j,
        9.35114006e-18+2.48813917e-17

In [ ]:
# from qiskit.algorithms import VQE
# from qiskit.chemistry.algorithms import VQEAdapt
# from IPython.display import display, clear_output

# # Initial Parameters of Ansatz
# try:
#     initial_point = [0.01] * len(ansatz.ordered_parameters)
# except:
#     initial_point = [0.01] * ansatz.num_parameters
    
# # algorithm = VQE(ansatz = ansatz, 
# #                 optimizer = optimizer, 
# #                 initial_point = initial_point, 
# #                 quantum_instance = backend)
# # #                 callback = callback)

# algorithm = VQEAdapt(qubit_operator, 
#                      var_form_base = ansatz, 
#                      optimizer = optimizer, 
#                      initial_point = initial_point, 
#                      quantum_instance = backend)
# #                 callback = callback)

# result = algorithm.compute_minimum_eigenvalue(qubit_operator)
# print(result)

## Exact Energy

In [19]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
# print(result_exact)

Exact electronic energy -1.0897823963487436
